In [193]:
# csce478    Programming Assignment 4     Fall 2020                                   Due: Nov 17 11am
# Linear Support Vector Machine & Principle Component Analysis
# Contributers: Chungsum Kim, Devn Steiner, Jesse Reyes Cortes

#imported packages
import pandas as pd
import numpy as np
import math
import copy
import itertools
from random import sample 
import nltk
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from random import seed
seed(1)
from random import random

# helper functions
def accuracy(x,y):
    count = 0
    for i in range(0, len(x)):
        if(x[i]==0 and 0 == y[i]):
            count = count + 1
        elif(x[i]==1 and 0 == y[i]):
            count = count + 1
        elif(x[i]==2 and y[i]==2):
            count = count + 1
    
    return count/len(x)


def confusion_matrix(x,y):
    TN = 0
    FP = 0
    FN = 0
    TP = 0
    for i in range(0, len(y)):
        if(x[i]==0 and y[i] <= .5):
            TN = TN + 1
        elif(x[i]==0 and .5 < y[i]):
            FP = FP + 1
        elif(x[i]==1 and y[i] <= .5):
            FN = FN + 1
        elif(x[i]==1 and .5 < y[i]):
            TP = TP + 1

    return [[TN, FP],[FN, TP]]


def adaptive_learning_rate(t_0, t_1, iteration):
    return t_0/(iteration + t_1)


def kFoldHelper(X,Y, k):
    
    kData = []
    
    x = np.array_split(X,k)
    y = np.array_split(Y,k)
    
    for i in range(0, k):
        x_train = np.array([])
        y_train = np.array([])
        x_test = x[i]
        y_test = y[i]
        for j in range(0,k):
            if i != j:
                if(len(x_train)==0):
                    x_train = x[j]
                else:
                    x_train = np.concatenate((x_train,x[j]))
                y_train = np.concatenate((y_train, y[j]))
            
        kData.append([x_train,y_train,x_test,y_test])
    
    return kData


def t_sv(X,w,b):
        
    t = np.array([])
        
    for j in range(0, len(X)):
        if np.dot(np.transpose(w), X[j])+b >= 1 :
            t = np.append(t, [1])
        else:
            t = np.append(t, [0])
                
    return t


def cost_function_svm(C,w,b, t, x_sv):
    
    wt = w.reshape((1,2))
    
    x_sv = x_sv.reshape(120,1)
    
        
    j = (1/2) * np.matmul(np.transpose(w), w) + C * ((np.sum(1-np.matmul(x_sv, wt))) - b * np.sum(t))
        
    return j


def decision_boundary_support_vectors(svm_clf, X):
    
    xmin, xmax = X.min() - 1, X.max() + 1
    
    w = svm_clf.coef_[0]
    b = svm_clf.intercept_[0]

    # At the decision boundary, w1*x1 + w2*x2 + b = 0
    # => x2 = -(b + w1* x1)/w1
    x1 = np.linspace(xmin, xmax, 100)
    
    decision_boundary = -(b + w[0]*x1)/w[1]

    shifting_factor_for_margin = 1/w[1]
    upper_margin = decision_boundary + shifting_factor_for_margin
    lower_margin = decision_boundary - shifting_factor_for_margin

    svs = svm_clf.support_vectors_
    plt.scatter(svs[:, 0], svs[:, 1], s=200, facecolors='g', label="Support Vectors")
    plt.plot(x1, decision_boundary, "k-", linewidth=2)
    plt.plot(x1, upper_margin, "k--", linewidth=2)
    plt.plot(x1, lower_margin, "k--", linewidth=2)



In [194]:
# Part A : Linear Support Vector Machine (SVM)

# A.1 implement Linear_SVC model class for performing binary classification

class Linear_SVC:
    def __init__(self, C=1, max_iter=100, tol=None, learning_rate="constant",learning_rate_init=0.001, t_0=1, t_1=1000, early_stopping=False, validation_fraction=0.1, **kwargs):
        self.C = C
        self.max_iter = max_iter
        self.tol = tol
        self.learning_rate = learning_rate
        self.learning_rate_init = learning_rate_init
        self.t_0 = t_0
        self.t_1 = t_1
        self.early_stopping = early_stopping
        self.validation_fraction = validation_fraction
        
        
    def fit(self, X, Y):
        self.X = X
        self.Y = Y
        
        # calculate the weight vector w with length of one sample with random small number
        self.w = np.array([])
        for i in range(0, len(self.X[0])):
            self.w = np.append(self.w, [random()])
            
        print(self.w)
        
        # intercept bias b originally set to 0
        self.b = 0
        
        
        
        for i in range(0, self.max_iter):
            
            t = t_sv(self.X, self.w, self.b)
            
            x_sv = t * (np.matmul(self.X, self.w) + self.b)
            
            # early stopping
            if abs(cost_function_svm(self.C, self.w, self.b, t, x_sv)) < self.tol:
                break
                
            
            grad_w = self.w - self.C * np.sum(x_sv)
            grad_b = -self.C * np.sum(t)
            
            self.w = self.w - grad_w
            self.b = self.b - grad_b
        
        self.intercept = np.array([self.b]) 
        self.coef = np.array([self.w]) 
        self.support_vectors = np.array([x_sv])
        
        return self
            
        
    def predict(self, X):
        
        print(X)
        
        
        return self.Y
        
        
        
        
        
        

In [195]:
# Binary Classification using Linear_SVC Classifier

# A.2 read the iris data, X features: petal length & petal width, recode binary target such that Iris-Virginica is 1 or 0
data = load_iris()

pl = data["data"][:,2]
pw = data["data"][:,3]
X = np.zeros((len(pl), 2))
X[:,0] = pl
X[:,1] = pw

Y = np.where(data['target'] == 2, 1, 0)

In [196]:
# A.3 partition data into train & test set (80% - 20%)
def partition(X, Y, t):     
    test_size = math.ceil(len(Y)*t)
    training_size = len(Y)-test_size
    training_data = X[0:training_size][:]
    testing_data = X[training_size+1:len(Y)] [:]
    training_vector = Y[0:training_size]
    testing_vector = Y[training_size+1:len(Y)]
    
    return training_data, testing_data, training_vector, testing_vector

In [197]:
# A.4 Hyper-parameter training(kFold - C, learning_rate, max_iter, tol)

#testing params
Cs = [.1,1]
learning_rate_inits = [.001, .1]
max_iters = [100, 1000]
tol = [.0001, .01]
learning_rates = ["constant", "adaptive"]

data = kFoldHelper(X, Y, 5)

dict_svm = {"C":[], "learn rate init":[], "max iter":[], "tol":[], "learn rate":[], "accuracy":[], "confusion matrix":[]}

for c in Cs:
    for lri in learning_rate_inits:
        for mi in max_iters:
            for t in tol:
                for lr in learning_rates:
                    y_hat = np.array([])
                    for d in data:
                        (print("Searning . . ."))
                        
                        x_train = d[0]
                        y_train = d[1]
                        x_test = d[2]
                        y_test = d[3]
                                                
                        model = Linear_SVC(C=c,max_iter=mi,tol=t,learning_rate=lr,learning_rate_init=lri,early_stopping=True)
                        
                        model.fit(x_train,y_train)
                        
                        y_hat = np.append(y_hat, model.predict(x_test))
                        
                        
                    #update dict
                    dict_svm["C"].append(c)
                    dict_svm["learn rate init"].append(lri)
                    dict_svm["max iter"].append(mi)
                    dict_svm["tol"].append(t)
                    dict_svm["learn rate"].append(lr)
                    dict_svm["accuracy"].append(accuracy(Y,y_hat))
                    dict_svm["confusion matrix"].append(confusion_matrix(Y,y_hat))
                        
                        



Searning . . .
[0.13436424 0.84743374]
[[1.4 0.2]
 [1.4 0.2]
 [1.3 0.2]
 [1.5 0.2]
 [1.4 0.2]
 [1.7 0.4]
 [1.4 0.3]
 [1.5 0.2]
 [1.4 0.2]
 [1.5 0.1]
 [1.5 0.2]
 [1.6 0.2]
 [1.4 0.1]
 [1.1 0.1]
 [1.2 0.2]
 [1.5 0.4]
 [1.3 0.4]
 [1.4 0.3]
 [1.7 0.3]
 [1.5 0.3]
 [1.7 0.2]
 [1.5 0.4]
 [1.  0.2]
 [1.7 0.5]
 [1.9 0.2]
 [1.6 0.2]
 [1.6 0.4]
 [1.5 0.2]
 [1.4 0.2]
 [1.6 0.2]]
Searning . . .
[0.76377462 0.25506903]


<ipython-input-193-09d065fcadcf>:100: RuntimeWarning: overflow encountered in matmul
  j = (1/2) * np.matmul(np.transpose(w), w) + C * ((np.sum(1-np.matmul(x_sv, wt))) - b * np.sum(t))
<ipython-input-193-09d065fcadcf>:100: RuntimeWarning: invalid value encountered in double_scalars
  j = (1/2) * np.matmul(np.transpose(w), w) + C * ((np.sum(1-np.matmul(x_sv, wt))) - b * np.sum(t))


[[1.6 0.2]
 [1.5 0.4]
 [1.5 0.1]
 [1.4 0.2]
 [1.5 0.2]
 [1.2 0.2]
 [1.3 0.2]
 [1.4 0.1]
 [1.3 0.2]
 [1.5 0.2]
 [1.3 0.3]
 [1.3 0.3]
 [1.3 0.2]
 [1.6 0.6]
 [1.9 0.4]
 [1.4 0.3]
 [1.6 0.2]
 [1.4 0.2]
 [1.5 0.2]
 [1.4 0.2]
 [4.7 1.4]
 [4.5 1.5]
 [4.9 1.5]
 [4.  1.3]
 [4.6 1.5]
 [4.5 1.3]
 [4.7 1.6]
 [3.3 1. ]
 [4.6 1.3]
 [3.9 1.4]]
Searning . . .
[0.49543509 0.44949106]
[[3.5 1. ]
 [4.2 1.5]
 [4.  1. ]
 [4.7 1.4]
 [3.6 1.3]
 [4.4 1.4]
 [4.5 1.5]
 [4.1 1. ]
 [4.5 1.5]
 [3.9 1.1]
 [4.8 1.8]
 [4.  1.3]
 [4.9 1.5]
 [4.7 1.2]
 [4.3 1.3]
 [4.4 1.4]
 [4.8 1.4]
 [5.  1.7]
 [4.5 1.5]
 [3.5 1. ]
 [3.8 1.1]
 [3.7 1. ]
 [3.9 1.2]
 [5.1 1.6]
 [4.5 1.5]
 [4.5 1.6]
 [4.7 1.5]
 [4.4 1.3]
 [4.1 1.3]
 [4.  1.3]]
Searning . . .
[0.65159297 0.78872335]
[[4.4 1.2]
 [4.6 1.4]
 [4.  1.2]
 [3.3 1. ]
 [4.2 1.3]
 [4.2 1.2]
 [4.2 1.3]
 [4.3 1.3]
 [3.  1.1]
 [4.1 1.3]
 [6.  2.5]
 [5.1 1.9]
 [5.9 2.1]
 [5.6 1.8]
 [5.8 2.2]
 [6.6 2.1]
 [4.5 1.7]
 [6.3 1.8]
 [5.8 1.8]
 [6.1 2.5]
 [5.1 2. ]
 [5.3 1.9]
 [5.5 2.1]
 [5.  2

IndexError: index 150 is out of bounds for axis 0 with size 150

In [187]:
# A.5 train model using optimal values for hyperparameters on test data, report accuracy and test confusion matrix
print(pd.DataFrame(dict_svm))

Empty DataFrame
Columns: [C, learn rate init, max iter, tol, learn rate, accuracy, confusion matrix]
Index: []


In [99]:
# A.6 plot the learning curve NOT COMPLETED
X = data.drop(columns='quality')
X = (X - X.mean())/X.std() ## standardized data
Y = data['quality']
model = Linear_Regression()
model.fit(X, Y, learning_rate=.01,lambd=1, regularizer='l2')
model.plot_curve()


AttributeError: 'list' object has no attribute 'drop'

In [100]:
# A.7 plot the decision boundary and show support vectors using “decision_boundary_support_vectors”
# https://github.com/rhasanbd/Support-Vector-Machine-Classifier-Beginners-Survival-Kit/blob/master/Support%20Vector%20Machine-1-Linearly%20Separable%20Data.ipynb
# Draw a scatter plot

# need to figure out what 
svm_clf = []

decision_boundary_support_vectors(svm_clf, X)

AttributeError: 'list' object has no attribute 'coef_'

In [61]:
# A.8 implement early stopping in the "fit" method of the Linear_SVC model, generate early stopping code


# this implementation was integrated into the Linear_SVM model A.1


In [ ]:
# Part B: Principle Component Analysis

# B.9 using matplotlib.pyploy "imread" function read the image as a 2D matrix


In [ ]:
# B.10 implement steps of eigendecomposition based PCA on X



In [ ]:
# B.11 find top k eigenvectors and create eigen vector matrix using top k eigenvectors


In [ ]:
# B.12 finally project mean centered data on the k top eigenvectors


In [ ]:
# B.13 reconstruct data matrix by taking dot product between projected data and transpose of top K eigenvec matrix


In [ ]:
# B.14 compute reconstruction error between mean centered data matrix X and reconstructed data matrix


In [ ]:
# B.15 perform steps 11-14 for k: 10, 30, 50, 100, 500
# for each k reconstruct image & print the value of k and reconstruction error